In [4]:
from qiskit import *
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, Aer, execute
provider = IBMQ.load_account()
from pprint import pprint as pp

ibmqfactory.load_account:WARNING:2020-12-26 19:38:26,628: Credentials are already in use. The existing account in the session will be replaced.


In [9]:
def binarise(inp, length):
    ret = []
    for j in range(length):
        ret.append(inp%2)
        inp //= 2
    return ret[::-1]

def qRAM(qc, address, add_reg, problem, in_reg, ancilla_reg, Mode):
    add_leng = len(add_reg)
    add = binarise(address, add_leng)
    in_leng = len(in_reg)
    for j in range(add_leng):
        if add[j] == 0:
            qc.x(add_reg[j])
    targets = []
    for j in range(in_leng):
        if problem[j] == 1:
            targets.append(j)
    chain_4CT(qc, add_reg, targets, ancilla_reg, in_reg)
    for j in range(add_leng):
        if add[j] == 0:
            qc.x(add_reg[j])
    qc.barrier()
    
def chain_4CT(qc, control_reg, targets, ancilla_reg, in_reg):
    qc.toffoli(control_reg[3], control_reg[2], ancilla_reg[0])
    qc.toffoli(control_reg[1], ancilla_reg[0], ancilla_reg[1])
    qc.toffoli(control_reg[0], ancilla_reg[1], ancilla_reg[2])
    for target in targets:
        qc.cx(ancilla_reg[2], in_reg[target])
    qc.toffoli(control_reg[0], ancilla_reg[1], ancilla_reg[2])
    qc.toffoli(control_reg[1], ancilla_reg[0], ancilla_reg[1])
    qc.toffoli(control_reg[3], control_reg[2], ancilla_reg[0])
    
def applyingQRAM(qc, add_reg, problemSet, in_reg, ancilla_reg, Mode):
    for j in range(16):
        qRAM(qc, j, add_reg, problemSet[j], in_reg, ancilla_reg, Mode)
        
def conditionalCounter(qc, count_reg, ancilla_reg, control, Mode):
    qc.mct(control, count_reg[1], ancilla_reg, mode=Mode)
    qc.mct(control[:-1], count_reg[1], ancilla_reg, mode=Mode)
    
        
def oracle(qc, in_reg, nonSolvables, count_reg, out_reg, ancilla_reg, Mode):
    controls = [[] for j in nonSolvables]
    i = 0
    for nonSolvable in nonSolvables:
        for num in nonSolvable:
            controls[i].append(in_reg[num])
        controls[i].append(count_reg[0])
        i+=1
        
    for j in range(24):
        conditionalCounter(qc, count_reg, ancilla_reg, controls[j], Mode)
        
    qc.cx(count_reg[0], out_reg)
    qc.cx(count_reg[1], out_reg)
        
def diffusion(qc, add_reg, ancilla_reg, Mode):
    qc.h(add_reg)
    qc.x(add_reg)
    qc.h(add_reg[0])
    qc.mct(add_reg[1:], add_reg[0], ancilla_reg, mode=Mode)
    qc.h(add_reg[0])
    qc.x(add_reg)
    qc.h(add_reg)
    
def Grover(qc, add_reg, problemSet, nonSolvables, in_reg, count_reg, out_reg, ancilla_reg, Mode, iters=1):
    for trial in range(iters):
        applyingQRAM(qc, add_reg, problemSet, in_reg, ancilla_reg, Mode)
        oracle(qc, in_reg, nonSolvables, count_reg, out_reg, ancilla_reg, Mode)
        applyingQRAM(qc, add_reg, problemSet, in_reg, ancilla_reg, Mode)
        diffusion(qc, add_reg, ancilla_reg, Mode)

In [10]:
problem_set = \
    [[['0', '2'], ['1', '0'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '3']],
    [['0', '0'], ['1', '1'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],    
    [['0', '0'], ['0', '1'], ['1', '1'], ['1', '3'], ['3', '2'], ['3', '3']],
    [['0', '2'], ['1', '0'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
    [['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '1'], ['3', '3']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']], 
    [['0', '0'], ['0', '3'], ['1', '2'], ['2', '2'], ['2', '3'], ['3', '0']],
    [['0', '3'], ['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '3'], ['2', '1'], ['2', '3'], ['3', '0']],
    [['0', '1'], ['0', '3'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '2']],
    [['0', '0'], ['1', '3'], ['2', '0'], ['2', '1'], ['2', '3'], ['3', '1']],
    [['0', '1'], ['0', '2'], ['1', '0'], ['1', '2'], ['2', '2'], ['2', '3']],
    [['0', '3'], ['1', '0'], ['1', '3'], ['2', '1'], ['2', '2'], ['3', '0']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '3'], ['3', '0'], ['3', '1']],
    [['0', '1'], ['1', '0'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '1']]]
def problemTranslator(problem_set):
    ProblemSet = []
    for problem in problem_set:
        temp = []
        for ij in problem:
            temp.append(int(ij[0])*4 + int(ij[1]))
        ProblemSet.append(temp)
    # pp(problemSet)
    problemSet = [[0]*16 for j in range(16)]

    for i in range(16):
        for j in ProblemSet[i]:
            problemSet[i][j] = 1
    return problemSet

perms = ['abcd', 'abdc', 'acbd', 'acdb', 'adbc', 'adcb',
         'bacd', 'badc', 'bcad', 'bcda', 'bdac', 'bdca',
         'cabd', 'cadb', 'cbad', 'cbda', 'cdab', 'cdba',
         'dabc', 'dacb', 'dbac', 'dbca', 'dcab', 'dcba'
        ]
def makeMatrix(strings):
    returnedList = [ ]
#     print(strings)
    for string in strings:
        if string == 'a':
            returnedList += [1,0,0,0]
        elif string == 'b':
            returnedList += [0,1,0,0]
        elif string == 'c':
            returnedList += [0,0,1,0]
        elif string == 'd':
            returnedList += [0,0,0,1]
#         else:
    return returnedList

def makeNonSolvableList(perms):
    NonSolvables = [[j for j in range(len(k)) if k[j] == 1]for k in [makeMatrix(perm) for perm in perms]]
    return NonSolvables

In [11]:
problemSet = problemTranslator(problem_set)
nonSolvables = makeNonSolvableList(perms)
add_reg = QuantumRegister(4, name='address')
in_reg = QuantumRegister(16, name='input')
count_reg = QuantumRegister(2, name='counter')
ancilla_reg = QuantumRegister(5, name='ancilla')
out_reg = QuantumRegister(1, name='out')
cbit = ClassicalRegister(4, name='cbits')
qc = QuantumCircuit(add_reg, in_reg, count_reg, out_reg, ancilla_reg, cbit)

qc.h(add_reg)
qc.x(out_reg)
qc.h(out_reg)
Mode = 'basic'

problemSet = problemTranslator(problem_set)
nonSolvables = makeNonSolvableList(perms)

Grover(qc, add_reg, problemSet, nonSolvables, in_reg, count_reg, out_reg, ancilla_reg, Mode)
qc.measure(add_reg[:], cbit[:4])

backend = BasicAer.get_backend('qasm_simulator')
shots = 1024
results = execute(qc, backend=backend, shots=shots).result()
answer = results.get_counts()
plot_histogram(answer)

BasicAerError: 'Number of qubits 28 is greater than maximum (24) for "qasm_simulator".'